# Wordcount i SQL

I denne notebook løser vi wordcount problemet ved at lave en SparkSQL-tabel og skrive forespørgsler imod denne.

In [1]:
# Vi starter med at indlæse data
nytaar=spark.sparkContext.textFile("gs://big-data-course-datasets/nytaar/")

Data består af relativt lange sætninger. Det bliver lettere at skrive sql forespørgslerne hvis vi kun har et ord pr række.

In [4]:
nytaar.take(1)

['For min familie og for mig selv blev dette år skelsættende ved min fader kong Frederiks sygdom og død. Den dybe sorg, der ramte os, følte vi, at hele folket tog del i, og jeg kan ikke begynde denne min første nytårshilsen uden at bringe en tak for al den varme og sympati, som blev prins Henrik og mig, men ikke mindst min moder, dronning Ingrid, til del.']

In [8]:
nytaar.flatMap(lambda x: x.replace("."," ").replace(","," ").lower().split()).take(10)

['for', 'min', 'familie', 'og', 'for', 'mig', 'selv', 'blev', 'dette', 'år']

In [12]:
word_rdd=nytaar.flatMap(lambda x: x.replace("."," ").lower().split())

**word_rdd** er nu en RDD af enkelt ord. Vi kan lave dette om til en tabel med én kolonne, som indeholder et enkelt ord.

In [31]:
from pyspark.sql import Row

word_df=word_rdd.map(lambda x: Row(word=x)).toDF()
word_df.registerTempTable("words")

In [32]:
word_df.show()

+------------+
|        word|
+------------+
|         for|
|         min|
|     familie|
|          og|
|         for|
|         mig|
|        selv|
|        blev|
|       dette|
|          år|
|skelsættende|
|         ved|
|         min|
|       fader|
|        kong|
|   frederiks|
|      sygdom|
|          og|
|         død|
|         den|
+------------+
only showing top 20 rows



In [24]:
spark.sql("SELECT * FROM words LIMIT 10").toPandas()

,word
0,for
1,min
2,familie
3,og
4,for
5,mig
6,selv
7,blev
8,dette
9,år


Det er ikke så kompliceret at skrive en forespørgsel, som finder de mest almindelige ord i talerne.

In [26]:
spark.sql("SELECT word, count(*) AS c FROM words GROUP BY word ORDER BY c DESC LIMIT 10").toPandas()

,word,c
0,og,2368
1,det,1730
2,i,1556
3,at,1463
4,vi,1452
5,er,1306
6,for,1160
7,til,1009
8,de,850
9,som,847


Og tilsvarende kan vi filtrere på ordene "prins" og "prinsesse".

In [29]:
spark.sql("""SELECT word, count(*) AS c 
FROM words 
WHERE LOWER(word) LIKE 'prins' OR LOWER(word) LIKE 'prinsesse' 
GROUP BY word""").toPandas()

,word,c
0,prins,38
1,prinsesse,16
